# CELLA 1 - SETUP ENV & PATHS

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Assuming you upload the folder 'ingegneria-dati-hw6' to the root of your Drive
    # If you put it in a subfolder, modify this path accordingly
    BASE_PATH = Path("/content/drive/MyDrive/ingegneria-dati-hw6")
    
    # Add the project directory to sys.path so we can import from src
    sys.path.append(str(BASE_PATH))
else:
    # Running locally
    BASE_PATH = Path("..")
    # Add the project root to sys.path to allow importing from src
    sys.path.append(str(BASE_PATH.resolve()))

print(f"Base path set to: {BASE_PATH}")


# CELLA 2 - IMPORT

In [ ]:
from src.profiling import profile_dataframe


# CELLA 3 - CARICAMENTO DATI

In [ ]:
df_cl = pd.read_csv(
    BASE_PATH / "data/raw/craigslist/vehicles.csv"
)

df_uc = pd.read_csv(
    BASE_PATH / "data/raw/us_used_cars/used_cars_data.csv",
    dtype={"dealer_zip": str}
)


# CELLA 4 - PROFILING

In [ ]:
profile_cl = profile_dataframe(df_cl)
profile_uc = profile_dataframe(df_uc)

profile_cl.head(10)


# ANALISI FEATURES E CLASSIFICAZIONE (Homework 6)

Basandoci sui risultati del profiling, classifichiamo le colonne come richiesto:

### 1. DA ELIMINARE (Drop)
- **county** (100% null): Colonna vuota.
- **size** (~72% null): Troppi dati mancanti, feature poco informativa.

### 2. DA MANTENERE (Keep)
- **manufacturer**: Fondamentale per l'identificazione.
- **title_status**: Bassa percentuale di null (1.9%), indicatore importante della qualità del veicolo.
- **VIN**: Chiave primaria per il Ground Truth.

### 3. DA IMPUTARE (Impute)
- **cylinders**, **condition**, **drive**, **paint_color**, **type**: Hanno null tra il 20% e il 40%. Useremo imputazione 'unknown'.


In [ ]:
# Salvataggio risultati del profiling
profile_cl.to_csv(
    BASE_PATH / "data/profiling/craigslist_profile.csv"
)

profile_uc.to_csv(
    BASE_PATH / "data/profiling/us_used_cars_profile.csv"
)


# CELLA 5 - Definizione Schema Mediato e Allineamento

In [ ]:
# 1. Definizione dello Schema Mediato (Punto 2 dell'Homework)
# Mappiamo le colonne di US Used Cars (df_uc) per farle coincidere con Craigslist (df_cl)
mapping_uc = {
    'vin': 'VIN',
    'make_name': 'manufacturer',
    'body_type': 'type',
    'exterior_color': 'paint_color',
    'drive_type': 'drive'
}

# Derivazione title_status per df_uc (usando 'salvage' se presente)
if 'salvage' in df_uc.columns:
    # Se salvage e' True -> 'salvage', altrimenti 'clean'
    # Assumiamo che i null in salvage siano 'clean' per sicurezza o 'unknown'
    df_uc['title_status'] = df_uc['salvage'].fillna(False).apply(lambda x: 'salvage' if x else 'clean')
else:
    # Fallback sicuro se salvage non esiste
    df_uc['title_status'] = 'clean'

# 2. Allineamento (Punto 3 dell'Homework)
# Selezioniamo le colonne utili per il Record Linkage e il Ground Truth
# AGGIUNTO: title_status
common_columns = ['VIN', 'manufacturer', 'type', 'paint_color', 'condition', 'cylinders', 'drive', 'title_status']

# Applichiamo il mapping e filtriamo le colonne
df_cl_aligned = df_cl[df_cl.columns.intersection(common_columns)].copy()
# Nota: title_status ora esiste in df_uc, quindi verra' incluso
df_uc_mapped = df_uc.rename(columns=mapping_uc)
df_uc_aligned = df_uc_mapped[df_uc_mapped.columns.intersection(common_columns)].copy()

print("Sorgenti allineate allo schema mediato.")


# CELLA 6 - Pulizia VIN e Creazione Ground Truth

In [ ]:
def clean_vin(df):
    if 'VIN' in df.columns:
        # Pulizia: maiuscolo, rimosione spazi e caratteri non alfanumerici
        df['VIN'] = df['VIN'].astype(str).str.upper().str.replace(r'[^A-Z0-9]', '', regex=True)
        # Sostituzione valori nulli o non validi
        df['VIN'] = df['VIN'].replace(['NAN', 'NONE', ''], np.nan)
    return df

df_cl_aligned = clean_vin(df_cl_aligned)
df_uc_aligned = clean_vin(df_uc_aligned)

# 3. Generazione Ground Truth (Punto 4.A)
# Troviamo i record che corrispondono esattamente tramite VIN
ground_truth = pd.merge(
    df_cl_aligned[['VIN']].dropna(), 
    df_uc_aligned[['VIN']].dropna(), 
    on='VIN', 
    how='inner'
).drop_duplicates()

print(f"Ground Truth generato con {len(ground_truth)} coppie basate su VIN.")


# CELLA 7 - Pre-processing Categoriche e Salvataggio

In [ ]:
def final_prep(df):
    # Rimuoviamo colonne inutili emerse dal profiling
    # AGGIUNTO: size, county
    cols_to_drop = [c for c in ['county', 'size'] if c in df.columns]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
    
    # Per il Record Linkage, e' meglio avere stringhe coerenti invece di NaN
    cols_to_fill = df.columns.difference(['VIN'])
    df[cols_to_fill] = df[cols_to_fill].fillna('unknown').astype(str).str.lower()
    return df

df_cl_final = final_prep(df_cl_aligned)
df_uc_final = final_prep(df_uc_aligned)

# Salvataggio dei dataset pronti per il Record Linkage (Punto 4.B e successivi)
df_cl_final.to_csv(BASE_PATH / "data/processed/cl_ready.csv", index=False)
df_uc_final.to_csv(BASE_PATH / "data/processed/uc_ready.csv", index=False)
ground_truth.to_csv(BASE_PATH / "data/ground_truth/ground_truth_vin.csv", index=False)
print("File salvati in data/processed e data/ground_truth")